In [1]:
!pip install tensorflow


In [2]:
!pip install pandas
!pip install scikit-learn
!pip install keras


In [6]:
import pandas as pd
data=pd.read_csv('/content/water.csv')
data.head()

,Route,Residence Type,Volume Required,Time of Day,Season,Flow Rate,Tank Volume,Valve 1,Valve 2,Valve 3,Valve 4,Valve 5
0,2,residential,185,morning,dry,0.650514,77,93,49,37,83,55
1,3,institution,388,afternoon,dry,0.817132,90,55,15,35,73,17
2,5,residential,134,morning,wet,0.743033,99,64,40,38,65,50
3,5,residential,226,morning,dry,0.766627,99,61,44,36,79,60
4,3,institution,264,afternoon,dry,0.921302,95,100,22,32,84,15


In [7]:
#preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('/content/water.csv')


label_encoder = LabelEncoder()
df['Season'] = label_encoder.fit_transform(df['Season'])
df['Time of Day'] = label_encoder.fit_transform(df['Time of Day'])


X = df[['Route', 'Volume Required', 'Time of Day', 'Season', 'Flow Rate', 'Tank Volume']]  # Features
y = df[['Valve 1', 'Valve 2', 'Valve 3', 'Valve 4', 'Valve 5']]  # Targets


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
!pip install tflite-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.2 MB/s eta 0:00:00


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build a simple neural network
model = Sequential()


model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 34508.4062 - val_loss: 415.6477
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 392.8269 - val_loss: 366.6485
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 369.7563 - val_loss: 370.1226
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 362.8608 - val_loss: 360.8397
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 356.3411 - val_loss: 417.7249
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 370.5645 - val_loss: 364.1565
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 350.7937 - val_loss: 358.6194
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 347.9960 - val_loss: 351.5255
Epoch 9/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 344.1498 - val_loss: 333.6136
Epoch 10/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 333.4095 - val_loss: 324.8886
Epoch 11/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 335.3105 - val_loss: 352.

In [14]:
# Save the trained model
model.save('valve_prediction_model.h5')


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load the test dataset (excluding valve values)
test_df = pd.read_csv('/content/testing2.csv')

# Convert categorical variables (Season, Time of Day) to numeric using LabelEncoder
label_encoder = LabelEncoder()
test_df['Season'] = label_encoder.fit_transform(test_df['Season'])
test_df['Time of Day'] = label_encoder.fit_transform(test_df['Time of Day'])

# Define features (X) for testing
X_test = test_df[['Route', 'Volume Required', 'Time of Day', 'Season', 'Flow Rate', 'Tank Volume']].values

# Load the saved model
model = tf.keras.models.load_model('valve_prediction_model.h5')

# Make predictions
y_pred = model.predict(X_test)

# Save the predictions along with input features into a new DataFrame
output_df = pd.DataFrame({
    'Route': test_df['Route'],
    'Volume Required': test_df['Volume Required'],
    'Time of Day': test_df['Time of Day'],
    'Season': test_df['Season'],
    'Flow Rate': test_df['Flow Rate'],
    'Tank Volume': test_df['Tank Volume'],
    'Predicted Valve 1': y_pred[:, 0],
    'Predicted Valve 2': y_pred[:, 1],
    'Predicted Valve 3': y_pred[:, 2],
    'Predicted Valve 4': y_pred[:, 3],
    'Predicted Valve 5': y_pred[:, 4]
})

# Save the predicted outputs to a CSV file
output_file_path = 'predicted_valve_outputs.csv'
output_df.to_csv(output_file_path, index=False)

print(f"Predicted outputs saved to {output_file_path}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted outputs saved to predicted_valve_outputs.csv
